In [1]:
import pandas as pd

%matplotlib inline

Data source
http://football-data.co.uk/englandm.php

Note
http://football-data.co.uk/notes.txt

In [2]:
# Season 2000/01 - 2017/18 (2000.csv, ..., 2017.csv)
# Predicted target MW #4 season 2017/18
# target_mw = 4
data_dir = './dataset/'
raw_data = []
for year in range(2000, 2018):
    raw_data.append(pd.read_csv(data_dir + str(year) + '.csv'))    

In [3]:
raw_data[0].head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,IWA,LBH,LBD,LBA,SBH,SBD,SBA,WHH,WHD,WHA
0,E0,19/08/00,Charlton,Man City,4,0,H,2,0,H,...,2.7,2.20,3.25,2.75,2.20,3.25,2.88,2.10,3.2,3.10
1,E0,19/08/00,Chelsea,West Ham,4,2,H,1,0,H,...,4.2,1.50,3.40,6.00,1.50,3.60,6.00,1.44,3.6,6.50
2,E0,19/08/00,Coventry,Middlesbrough,1,3,A,1,1,D,...,2.7,2.25,3.20,2.75,2.30,3.20,2.75,2.30,3.2,2.62
3,E0,19/08/00,Derby,Southampton,2,2,D,1,2,A,...,3.5,2.20,3.25,2.75,2.05,3.20,3.20,2.00,3.2,3.20
4,E0,19/08/00,Leeds,Everton,2,0,H,2,0,H,...,4.5,1.55,3.50,5.00,1.57,3.60,5.00,1.61,3.5,4.50


# SELECT FEATURES

In [4]:
selected_columns = ['HomeTeam','AwayTeam','FTHG','FTAG','FTR']
seasons = []
for data in raw_data:
    seasons.append(data[selected_columns])

In [5]:
seasons[17].tail(10)

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
30,Man City,Liverpool,0,0,NaN
31,Arsenal,Bournemouth,0,0,NaN
32,Brighton,West Brom,0,0,NaN
33,Everton,Tottenham,0,0,NaN
34,Leicester,Chelsea,0,0,NaN
35,Southampton,Watford,0,0,NaN
36,Stoke,Man United,0,0,NaN
37,Burnley,Crystal Palace,0,0,NaN
38,Swansea,Newcastle,0,0,NaN
39,West Ham,Huddersfield,0,0,NaN


# CREATE FEATURES

## AGGREGATED GOALS SCORED / CONCEDED (AHTGS, AATGS, AHTGC, AATGC)

Goals scored and conceded aggregated at the end of matchweek in each season, arranged by team and matchweek

In [6]:
# HTGS - Home Team Goal Scored
# ATGS = Away Team Goal Scored

# Create DataFrame for aggregated goals scored by each teams till each matchweeks
def get_agg_goals_scored(season):
    # Create  a dictonary with team names as keys
    teams = {}
    for i in season.groupby('HomeTeam').mean().T.columns:  # get team name as index
        teams[i] = []
    
    # Goals scored each matchweek by each team (as either Home Team or Away Team)
    for i in range(len(season)):
        HTGS = season.iloc[i]['FTHG']
        ATGS = season.iloc[i]['FTAG']
        teams[season.iloc[i].HomeTeam].append(HTGS)
        teams[season.iloc[i].AwayTeam].append(ATGS)
        
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    goals_scored = pd.DataFrame(data=teams, index=[i for i in range(1,(len(season) // 10) + 1)]).T
    goals_scored[0] = 0
    
    # Aggregate goals scored till each matchweek.
    for i in range(2,(len(season) // 10) + 1):
        goals_scored[i] = goals_scored[i] + goals_scored[i-1]
        
    return goals_scored


# HTGC - Home Team Goal Conceded
# ATGC = Away Team Goal Conceded

# Create DataFrame for aggregated goals conceded by each teams till each matchweeks
def get_agg_goals_conceded(season):
    # Create  a dictonary with team names as key
    teams = {}
    for i in season.groupby('HomeTeam').mean().T.columns:  # get team name as index
        teams[i] = []
    
    # Goals conceded each matchweek by each team (as either Home Team or Away Team)
    for i in range(len(season)):
        ATGC = season.iloc[i]['FTHG']
        HTGC = season.iloc[i]['FTAG']
        teams[season.iloc[i].HomeTeam].append(HTGC)
        teams[season.iloc[i].AwayTeam].append(ATGC)
        
    # Create a dataframe for goals conceded where rows are teams and cols are matchweek.
    goals_conceded = pd.DataFrame(data=teams, index=[i for i in range(1,(len(season) // 10) + 1)]).T
    goals_conceded[0] = 0
    
    # Aggregate goals conceded till each matchweek.
    for i in range(2,(len(season) // 10) + 1):
        goals_conceded[i] = goals_conceded[i] + goals_conceded[i-1]
        
    return goals_conceded

# Add aggregate goals scored and conceded of Home Team and Away Team before matchweek to gameplay_stat
# AHTGS - Aggreated Home Team Goal Scored
# AATGS - Aggreated Away Team Goal Scored
# AHTGC - Aggreated Home Team Goal Conceded
# AATGC - Aggreated Away Team Goal Conceded

def get_gss(season):
    AGS = get_agg_goals_scored(season)
    AGC = get_agg_goals_conceded(season)
    
    j = 0
    AHTGS = []
    AATGS = []
    AHTGC = []
    AATGC = []
    
    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        AHTGS.append(AGS.loc[ht][j])
        AATGS.append(AGS.loc[at][j])
        AHTGC.append(AGC.loc[ht][j])
        AATGC.append(AGC.loc[at][j])
        
        if ((i + 1) % 10) == 0:
            j = j + 1
            
    season['AHTGS'] = AHTGS
    season['AATGS'] = AATGS
    season['AHTGC'] = AHTGC
    season['AATGC'] = AATGC
    
    return season


# Apply to each season
for i, _ in enumerate(seasons):
    seasons[i] = get_gss(seasons[i])

/home/morn/ml/.tf/lib/python3.5/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/morn/ml/.tf/lib/python3.5/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/morn/ml/.tf/lib/python3.5/site-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

In [7]:
seasons[17].tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC
35,Southampton,Watford,0,0,NaN,3,5,2,3
36,Stoke,Man United,0,0,NaN,2,10,2,0
37,Burnley,Crystal Palace,0,0,NaN,4,0,4,6
38,Swansea,Newcastle,0,0,NaN,2,3,4,3
39,West Ham,Huddersfield,0,0,NaN,2,4,10,0


## GET RESPECTIVE POINTS (HTP, ATP)

In [8]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

def get_cuml_points(match_results, season):
    matchres_points = match_results.applymap(get_points)
    for i in range(2, (len(season) // 10) + 1):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column=0, loc=0, value=[0 * i for i in range(20)])
    return matchres_points

def get_match_results(season):
    # Create dictionary with team names as keys
    teams = {}
    for i in season.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    # the value corresponding to keys is a list containing the match result
    for i in range(len(season)):
        if season.iloc[i].FTR == 'H':
            teams[season.iloc[i].HomeTeam].append('W')
            teams[season.iloc[i].AwayTeam].append('L')
        elif season.iloc[i].FTR == 'A':
            teams[season.iloc[i].HomeTeam].append('L')
            teams[season.iloc[i].AwayTeam].append('W')
        else:
            teams[season.iloc[i].HomeTeam].append('D')
            teams[season.iloc[i].AwayTeam].append('D')
            
    return pd.DataFrame(data=teams, index=[i for i in range(1, (len(season) // 10) + 1)]).T

# HTP - Home Team Points
# ATP - Away Team Points

def get_agg_points(season):
    match_results = get_match_results(season)
    cum_pts = get_cuml_points(match_results, season)
    HTP = []
    ATP = []
    j = 0
    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])
        
        if ((i + 1) % 10) == 0:
            j = j + 1
            
    season.loc[:,'HTP'] = HTP
    season.loc[:,'ATP'] = ATP
    
    return season

# Apply to each season
for i, _ in enumerate(seasons):
    seasons[i] = get_agg_points(seasons[i])

/home/morn/ml/.tf/lib/python3.5/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/morn/ml/.tf/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
seasons[17].tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP
35,Southampton,Watford,0,0,NaN,3,5,2,3,5,5
36,Stoke,Man United,0,0,NaN,2,10,2,0,4,9
37,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,4,0
38,Swansea,Newcastle,0,0,NaN,2,3,4,3,4,3
39,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0,7


## GET TEAM FORM (HM1, HM2, HM3, AM1, AM2, AM3)

In [10]:
def get_form(season, num):          # the num th before
    form = get_match_results(season)
    form_final = form.copy()
    for i in range(num, (len(season) // 10) + 1):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1
    return form_final

def add_form(season, num):
    form = get_form(season, num)
    h = ['M' for i in range(num * 10)]    # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num * 10), len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        
        past = form.loc[ht][j]    # get past n results
        h.append(past[num - 1])   # 0 index is most recent
        
        past = form.loc[at][j]    # get past n results
        a.append(past[num - 1])   # 0 in dex is most recent
        
        if ((i + 1) % 10) == 0:
            j = j + 1
            
    season['HM' + str(num)] = h
    season['AM' + str(num)] = a
    
    return season

def add_form_df(season):
    season = add_form(season, 1)
    season = add_form(season, 2)
    season = add_form(season, 3)
    return season

# Apply to each season
for i, _ in enumerate(seasons):
    seasons[i] = add_form_df(seasons[i])

/home/morn/ml/.tf/lib/python3.5/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/morn/ml/.tf/lib/python3.5/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
# Rearranging columns
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'AHTGS', 'AATGS', 'AHTGC', 'AATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'AM1', 'AM2', 'AM3']

# Apply to each season
for i, _ in enumerate(seasons):
    seasons[i] = seasons[i][cols]

In [12]:
seasons[17].tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3
35,Southampton,Watford,0,0,NaN,3,5,2,3,5,5,D,W,D,D,W,D
36,Stoke,Man United,0,0,NaN,2,10,2,0,4,9,D,W,L,W,W,W
37,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,4,0,D,L,W,L,L,L
38,Swansea,Newcastle,0,0,NaN,2,3,4,3,4,3,W,L,D,W,L,L
39,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0,7,L,L,L,D,W,W


## GET LAST YEAR'S POSITION (HomeTeamLP, AwayTeamLP)

Get Last Year's Position

In [13]:
standings = pd.read_csv(data_dir + 'EPLStandings.csv')
standings.set_index(['Team'], inplace=True)
standings = standings.fillna(18)

def get_last(season, standings, year):
    home_team_lp = []
    away_team_lp = []
    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        home_team_lp.append(standings.loc[ht][year])
        away_team_lp.append(standings.loc[at][year])
    season['HomeTeamLP'] = home_team_lp
    season['AwayTeamLP'] = away_team_lp
    return season

# Apply to each season
for i, _ in enumerate(seasons):
    seasons[i] = get_last(seasons[i], standings, i)

In [14]:
seasons[17].tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,HomeTeamLP,AwayTeamLP
35,Southampton,Watford,0,0,NaN,3,5,2,3,5,5,D,W,D,D,W,D,8.0,17.0
36,Stoke,Man United,0,0,NaN,2,10,2,0,4,9,D,W,L,W,W,W,13.0,6.0
37,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,4,0,D,L,W,L,L,L,16.0,14.0
38,Swansea,Newcastle,0,0,NaN,2,3,4,3,4,3,W,L,D,W,L,L,15.0,18.0
39,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0,7,L,L,L,D,W,W,11.0,18.0


## Get MatchWeek (MW)

In [15]:
def get_mw(season):
    j = 1
    match_week = []
    for i in range(len(season)):
        match_week.append(j)
        if ((i + 1) % 10) == 0:
            j = j + 1
    season['MW'] = match_week
    return season

# Apply to each season
for i, _ in enumerate(seasons):
    seasons[i] = get_mw(seasons[i])

In [16]:
seasons[17].tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,HomeTeamLP,AwayTeamLP,MW
35,Southampton,Watford,0,0,NaN,3,5,2,3,5,5,D,W,D,D,W,D,8.0,17.0,4
36,Stoke,Man United,0,0,NaN,2,10,2,0,4,9,D,W,L,W,W,W,13.0,6.0,4
37,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,4,0,D,L,W,L,L,L,16.0,14.0,4
38,Swansea,Newcastle,0,0,NaN,2,3,4,3,4,3,W,L,D,W,L,L,15.0,18.0,4
39,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0,7,L,L,L,D,W,W,11.0,18.0,4


## FINAL DATAFRAME

In [17]:
gameplays = pd.concat(seasons, ignore_index=True)

In [18]:
gameplays.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,HomeTeamLP,AwayTeamLP,MW
6495,Southampton,Watford,0,0,NaN,3,5,2,3,5,5,D,W,D,D,W,D,8.0,17.0,4
6496,Stoke,Man United,0,0,NaN,2,10,2,0,4,9,D,W,L,W,W,W,13.0,6.0,4
6497,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,4,0,D,L,W,L,L,L,16.0,14.0,4
6498,Swansea,Newcastle,0,0,NaN,2,3,4,3,4,3,W,L,D,W,L,L,15.0,18.0,4
6499,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0,7,L,L,L,D,W,W,11.0,18.0,4


## Get Different Last Year's Position (DiffLP)

In [19]:
# Diff in last year positions
gameplays['DiffLP'] = gameplays['HomeTeamLP'] - gameplays['AwayTeamLP']

## Get Goal Difference

In [20]:
# Get Goal Difference
gameplays['HTGD'] = gameplays['AHTGS'] - gameplays['AHTGC']
gameplays['ATGD'] = gameplays['AATGS'] - gameplays['AATGC']

## Get Different Form Points (DiffFormPts)

In [21]:
# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

gameplays['HTFormPtsStr'] = gameplays['HM1'] + gameplays['HM2'] + gameplays['HM3']
gameplays['ATFormPtsStr'] = gameplays['AM1'] + gameplays['AM2'] + gameplays['AM3']

gameplays['HTFormPts'] = gameplays['HTFormPtsStr'].apply(get_form_points)
gameplays['ATFormPts'] = gameplays['ATFormPtsStr'].apply(get_form_points)

# Gets difference form point
gameplays['DiffFormPts'] = gameplays['HTFormPts'] - gameplays['ATFormPts']

In [22]:
gameplays.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,...,AwayTeamLP,MW,DiffLP,HTGD,ATGD,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,DiffFormPts
6495,Southampton,Watford,0,0,NaN,3,5,2,3,5,...,17.0,4,-9.0,1,2,DWD,DWD,5,5,0
6496,Stoke,Man United,0,0,NaN,2,10,2,0,4,...,6.0,4,7.0,0,10,DWL,WWW,4,9,-5
6497,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,4,...,14.0,4,2.0,0,-6,DLW,LLL,4,0,4
6498,Swansea,Newcastle,0,0,NaN,2,3,4,3,4,...,18.0,4,-3.0,-2,0,WLD,WLL,4,3,1
6499,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0,...,18.0,4,-7.0,-8,4,LLL,DWW,0,7,-7


In [23]:
# Scale HTP, ATP, HTGD, ATGD, DiffFormPts.
cols = ['HTP','ATP','HTGD','ATGD','DiffFormPts']
gameplays.MW = gameplays.MW.astype(float)

for col in cols:
    gameplays[col] = gameplays[col] / gameplays.MW

gameplays.tail()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,...,AwayTeamLP,MW,DiffLP,HTGD,ATGD,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,DiffFormPts
6495,Southampton,Watford,0,0,NaN,3,5,2,3,1.25,...,17.0,4.0,-9.0,0.25,0.5,DWD,DWD,5,5,0.00
6496,Stoke,Man United,0,0,NaN,2,10,2,0,1.00,...,6.0,4.0,7.0,0.00,2.5,DWL,WWW,4,9,-1.25
6497,Burnley,Crystal Palace,0,0,NaN,4,0,4,6,1.00,...,14.0,4.0,2.0,0.00,-1.5,DLW,LLL,4,0,1.00
6498,Swansea,Newcastle,0,0,NaN,2,3,4,3,1.00,...,18.0,4.0,-3.0,-0.50,0.0,WLD,WLL,4,3,0.25
6499,West Ham,Huddersfield,0,0,NaN,2,4,10,0,0.00,...,18.0,4.0,-7.0,-2.00,1.0,LLL,DWW,0,7,-1.75


In [24]:
gameplays.to_csv(data_dir + 'final_dataset.csv')